In [12]:
import numpy as np
DTYPE = np.float64
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import os
import pickle
import gzip
from tqdm import tqdm
try:
    import flory
    
except ImportError:
    print("Installing 'flory' temporarily...")
    !pip install flory --quiet
    import flory

from numba import jit

In [13]:
# Global concentrations
phi_global = np.array([0.3, 0.3, 0.4], dtype = DTYPE)

# For phi grid
step_size = 0.001
n_points = 100

Xs = np.arange(1, 1.09, 0.1)

tol = DTYPE(1e-4)

In [14]:
# Flory Huggins Free Energy function
def floryHuggins(phi:DTYPE, chi:np.array):
    part_1 = np.sum(phi*np.log(phi))
    part_2 = 0

    for i in range(len(phi)):
        for j in range(i+1, len(phi)):
            part_2 += chi[i][j]*phi[i]*phi[j]

    return part_1 + part_2

In [15]:
# General Function for computing mergers of compartments
# Returns the concentrations of components in the merged and the unmerged compartments
# Stores them in phi_in_kmerged and phi_in_kunmerged respectively.


def mergers(concs:np.array, vols:np.array, chis, merged_compartments:list):
    # Find the unmerged compartment(s)
    expected = len(vols)*(len(vols)+1)//2
    actual = np.sum(merged_compartments)
    unmerged_compartment = expected - actual

    # print(concs)
    # print(vols)
    # print(unmerged)

    # Compute the merged volumes, stored in variable eta_merged
    # subtract -1 from the merged_compartments idxs to maintain python idxing
    eta_merged = 0
    for compartment in merged_compartments:
        eta_merged += vols[compartment-1]
    # print(eta_merged)

    # Calculating the compositions of components in the merged compartment
    # Taking a simple weighted average
    phi_1merged = (vols[merged_compartments[0]-1]*concs[0, merged_compartments[0]-1] + vols[merged_compartments[1]-1]*concs[0, merged_compartments[1]-1])/eta_merged
    phi_2merged = (vols[merged_compartments[0]-1]*concs[1, merged_compartments[0]-1] + vols[merged_compartments[1]-1]*concs[1, merged_compartments[1]-1])/eta_merged
    phi_3merged = 1 - phi_1merged - phi_2merged
    # print(phi_1merged, phi_2merged, phi_3merged)
    # print(phi_1merged + phi_2merged + phi_3merged)

    phi_in_kmerged = [phi_1merged, phi_2merged, phi_3merged]
    phi_in_kunmerged = [concs[0, unmerged_compartment-1], concs[1, unmerged_compartment-1], concs[2, unmerged_compartment-1]]
    # print(phi_in_kmerged)
    # print(phi_in_kunmerged)

    F_merged = eta_merged*floryHuggins(phi_in_kmerged, chis) + vols[unmerged_compartment-1]*floryHuggins(phi_in_kunmerged, chis)
    # print(F_merged)

    return phi_in_kunmerged, phi_in_kmerged, eta_merged, unmerged_compartment

    

In [16]:
# Perturbs the initial guess
# Calculates free energy in slcies of eta_merged_perturbed for (phi_1merged, phi_2merged) per slice

def perturb_mergers(concs, vols, chis, phi_in_kunmerged, phi_in_kmerged, eta_merged, merged_compartments, unmerged_compartment, step_size: float, n_points: int):
    phi_1merged_perturbed = np.linspace(max(tol, phi_in_kmerged[0] - n_points*step_size), min(1-tol, phi_in_kmerged[0] + n_points*step_size), 2*n_points+1)
    phi_2merged_perturbed = np.linspace(max(tol, phi_in_kmerged[1] - n_points*step_size), min(1-tol, phi_in_kmerged[1] + n_points*step_size), 2*n_points+1)
    eta_merged_perturbed = np.linspace(max(tol, eta_merged - n_points*step_size), min(1-tol, eta_merged + n_points*step_size), 2*n_points+1)
    
    phi_1global = phi_global[0]
    phi_2global = phi_global[1]

    for eta_m in eta_merged_perturbed:
        if eta_m<0 or eta_m>1:
            continue
        eta_unmerged = 1 - eta_m # volume of the unmerged compartment

        accepted_phi_1merged = []
        accepted_phi_2merged = []
        accepted_Fs = []
        
        for phi_1m in phi_1merged_perturbed:
            for phi_2m in phi_2merged_perturbed:
                # concentrations in the unmerged compartments
                phi_1unmerged = (phi_1global - eta_m*phi_1m)/eta_unmerged
                phi_2unmerged = (phi_2global - eta_m*phi_2m)/eta_unmerged
                
                if phi_1unmerged<0 or phi_1unmerged>1 or phi_2unmerged<0 or phi_2unmerged>1:
                    continue  
                    

                phi_3m = 1 - phi_1m - phi_2m
                phi_3unmerged = 1 - phi_1unmerged - phi_2unmerged
                
                phi_in_kmerged = [phi_1m, phi_2m, phi_3m]
                phi_in_kunmerged =  [phi_1unmerged, phi_2unmerged, phi_3unmerged]

                # Compute the free energy for the perturbed configurations for eta_m as the merger volume
                F = eta_m*floryHuggins(phi_in_kmerged, chis) + eta_unmerged*floryHuggins(phi_in_kunmerged, chis)

                accepted_phi_1merged.append(phi_1m)
                accepted_phi_2merged.append(phi_2m)
                accepted_Fs.append(F)
                
        output_filepath = f"data/withVolFluctuations/phi_g{phi_global}/raw/X{X:.3f}/mergers/{merged_compartments}/volume_slices/"
        output_filename = f"eta_m{eta_m:.3f}.pkl"
        if not os.path.exists(output_filepath):
            os.makedirs(output_filepath)
        output_file = os.path.join(output_filepath, output_filename)
        data_to_save = {
            "eta_merged": eta_m,
            "phi_1merged": accepted_phi_1merged,
            "phi_2merged": accepted_phi_2merged,
            "F": accepted_Fs,
            "metadata": {
                "merged_compartments": merged_compartments,  # Which compartments were merged
                "unmerged_compartment": unmerged_compartment,  # The remaining compartment
                "initial_merged_concentration_guess": [phi_in_kmerged[0], phi_in_kmerged[1]] # initial guesses of the concentrations in the merged compartment. i.e. the values around which the perturbations take place.
            }
        }
        with gzip.open(output_file, "wb") as f:
            pickle.dump(data_to_save, f, protocol=pickle.HIGHEST_PROTOCOL)


In [17]:
# Plot the contours for best merger in each system
def plot_Contours(Xs):
    for X in tqdm(Xs):
        # Get the best merger
        input_filepath = f"data/noVolFluctuations/phi_g{phi_global}/analysis/X{X:.3f}"
        input_filename = f"best_merger.pkl"
        if not os.path.exists(input_filepath):
            os.makedirs(input_filepath)
        input_file = os.path.join(input_filepath, input_filename)
        
        with gzip.open(input_file, 'rb') as f:
            loaded_data = pickle.load(f)
        best_merger = loaded_data["best_merger"]
    
        # Load the best merger
        input_filepath = f"data/noVolFluctuations/phi_g{phi_global}/raw/X{X:.3f}/mergers/"
        input_filename = f"{best_merger}.pkl"
        input_file = os.path.join(input_filepath, input_filename)
        
        with gzip.open(input_file, "rb") as f:
            loaded_data = pickle.load(f)
        
        df_loaded = loaded_data["dataframe"]
    
    
        phi1_grid = np.linspace(min(df_loaded["phi_1merged"]), max(df_loaded["phi_1merged"]), 100)
        phi2_grid = np.linspace(min(df_loaded["phi_2merged"]), max(df_loaded["phi_2merged"]), 100)
        phi1_mesh, phi2_mesh = np.meshgrid(phi1_grid, phi2_grid)
        
        # Interpolate F values onto the grid
        F_grid = griddata(
            (df_loaded["phi_1merged"], df_loaded["phi_2merged"]), 
            df_loaded["F"], 
            (phi1_mesh, phi2_mesh), 
            method='cubic'
        )
        
        # Create the contour plot
        fig, ax = plt.subplots(figsize=(8, 6))
        contour = plt.contourf(phi1_mesh, phi2_mesh, F_grid, levels=1000, cmap = "RdYlBu_r")
        plt.colorbar(contour, label=r'$F\, (k_BT)$')
        
        # Initial Guess
        phi1_guess = loaded_data["metadata"]["initial_merged_concentration_guess"][0]
        phi2_guess = loaded_data["metadata"]["initial_merged_concentration_guess"][1]
        label = r"($\phi_{1, \text{merged}}^{in}, \phi_{2, \text{merged}}^{in}) =$" + f"({phi1_guess:.3f}, {phi2_guess:.3f})"
        ax.scatter(
            phi1_guess, phi2_guess, 
            color='magenta',
            marker = ".",
            linewidth=0.5,
            s=50, 
            label=label
        )
    
        # Find the minimum free energy point
        min_F_index = np.argmin(df_loaded["F"])
        min_phi1 = df_loaded["phi_1merged"].iloc[min_F_index]
        min_phi2 = df_loaded["phi_2merged"].iloc[min_F_index]
        min_F = df_loaded["F"].iloc[min_F_index]
        
        label = r"($\phi_{1, \text{merged}}^{\text{best}}, \phi_{2, \text{merged}}^{\text{best}}) =$" + f"({min_phi1:.3f}, {min_phi2:.3f})"
        ax.scatter(
            min_phi1, min_phi2, 
            color='lightgreen', 
            s=50,
            marker = ".",
            linewidth=0.5,
            label=label
        )
        
        # Add contour lines
        CS = plt.contour(phi1_mesh, phi2_mesh, F_grid, levels=40, colors='k', linewidths=0.25)
        
        phi1_d = phi1_grid[1]-phi1_grid[0]
        phi2_d = phi2_grid[1]-phi2_grid[0]
        ax.set_xlim([phi1_grid[0]-5*phi1_d, phi1_grid[-1]+5*phi1_d])
        ax.set_ylim([phi2_grid[0]-5*phi2_d, phi2_grid[-1]+5*phi2_d])
        
        
        ax.set_xlabel(r'$\phi_{1, \text{merged}}$', fontsize=12)
        ax.set_ylabel(r'$\phi_{2, \text{merged}}$', fontsize=12)
    
        title = f"X = {X:.3f}" + "\n" + r"$\Phi^{\text{global}}=$" + f"{phi_global}" +"\n" + f'Merged compartments: {loaded_data["metadata"]["merged_compartments"]}' + "\n" + r"$F_{\text{min}}=$" + f"{min_F:.3f}" 
        plt.title(title)
        plt.legend(loc = "lower left")
    
        fig.tight_layout()
    
        output_filepath = f"data/noVolFluctuations/phi_g{phi_global}/analysis/X{X:.3f}"
        output_filename = f"best_merger_contour.png"
        if not os.path.exists(output_filepath):
            os.makedirs(output_filepath)
        output_file = os.path.join(output_filepath, output_filename)
        
        plt.savefig(output_file, dpi=400)
        plt.close()

In [18]:
# Plot the variation in the free energy of the merger and the system as a function of X
def plot_FvsX(Xs):
    Fs_flory = []
    Fs_best_merger = []
    for X in tqdm(Xs):
        # Get the Flory's computation
        input_filepath = f"data/noVolFluctuations/phi_g{phi_global}/raw/X{X:.3f}/"
        input_filename = f"FLORY_3phase_solution.pkl"
        input_file = os.path.join(input_filepath, input_filename)
        
        with gzip.open(input_file, 'rb') as f:
            loaded_data = pickle.load(f)

        Fs_flory.append(loaded_data["flory_free_energy"])
        
        # Get the best merger
        input_filepath = f"data/noVolFluctuations/phi_g{phi_global}/analysis/X{X:.3f}"
        input_filename = f"best_merger.pkl"
        if not os.path.exists(input_filepath):
            os.makedirs(input_filepath)
        input_file = os.path.join(input_filepath, input_filename)
        
        with gzip.open(input_file, 'rb') as f:
            loaded_data = pickle.load(f)
        best_merger = loaded_data["best_merger"]
    
        # Load the best merger
        input_filepath = f"data/noVolFluctuations/phi_g{phi_global}/raw/X{X:.3f}/mergers/"
        input_filename = f"{best_merger}.pkl"
        input_file = os.path.join(input_filepath, input_filename)
        
        with gzip.open(input_file, "rb") as f:
            loaded_data = pickle.load(f)
        
        df_loaded = loaded_data["dataframe"]

        min_F_index = np.argmin(df_loaded["F"])
        min_phi1 = df_loaded["phi_1merged"].iloc[min_F_index]
        min_phi2 = df_loaded["phi_2merged"].iloc[min_F_index]
        min_F = df_loaded["F"].iloc[min_F_index]

        Fs_best_merger.append(min_F)

    fig, ax = plt.subplots(figsize=(4, 3))
    ax.plot(Xs, Fs_flory, label = r"$F_{\text{min}}^{\text{3-phase}}$")
    ax.plot(Xs, Fs_best_merger, label = r"$F_{\text{min}}^{\text{best merger}}$")
    ax.set_xlabel("X")
    ax.set_ylabel(r"$F \;(\text{in }k_BT)$")
    title = r"$\Phi^{\text{global}} = $" + f"{phi_global}" 
    plt.legend(loc = "best")
    plt.title(title)
    
    fig.tight_layout()

    
    output_filepath = f"data/noVolFluctuations/phi_g{phi_global}/analysis"
    output_filename = f"F_comparison.png"
    if not os.path.exists(output_filepath):
        os.makedirs(output_filepath)
    output_file = os.path.join(output_filepath, output_filename)
    
    plt.savefig(output_file, dpi =400)
    plt.close()


    F_diff = []
    for _ in list(zip(Fs_flory, Fs_best_merger)):
        F_diff.append(_[1] - _[0])

    fig, ax = plt.subplots(figsize = (4, 3))
    ylabel = r"$F_{\text{min}}^{\text{best merger}} - F_{\text{min}}^{\text{3-phase}}$"
    ax.plot(Xs, F_diff, marker = ".")
    ax.set_xlabel("X")
    ax.set_ylabel(ylabel)
    title = r"$\Phi^{\text{global}} = $" + f"{phi_global}" 
    plt.title(title)
    
    fig.tight_layout()

    output_filepath = f"data/noVolFluctuations/phi_g{phi_global}/analysis"
    output_filename = f"F_diff_comparison.png"
    if not os.path.exists(output_filepath):
        os.makedirs(output_filepath)
    output_file = os.path.join(output_filepath, output_filename)
    
    plt.savefig(output_file, dpi =400)
    plt.close()

In [19]:
# Plot the Euclidean distance between the guest and the best as a function of X
def plot_XvsD(Xs):
    ds = []
    for X in tqdm(Xs):
        # Get the best merger
        input_filepath = f"data/noVolFluctuations/phi_g{phi_global}/analysis/X{X:.3f}"
        input_filename = f"best_merger.pkl"
        if not os.path.exists(input_filepath):
            os.makedirs(input_filepath)
        input_file = os.path.join(input_filepath, input_filename)
        
        with gzip.open(input_file, 'rb') as f:
            loaded_data = pickle.load(f)
        best_merger = loaded_data["best_merger"]
    
        # Load the best merger
        input_filepath = f"data/noVolFluctuations/phi_g{phi_global}/raw/X{X:.3f}/mergers/"
        input_filename = f"{best_merger}.pkl"
        input_file = os.path.join(input_filepath, input_filename)
        
        with gzip.open(input_file, "rb") as f:
            loaded_data = pickle.load(f)
        
        df_loaded = loaded_data["dataframe"]
    
        # Plot the best merger
        initial_merged_concentration_guess = loaded_data["metadata"]["initial_merged_concentration_guess"]
    
        # concentrations with the lowest free energies
        min_F_index = np.argmin(df_loaded["F"])
        min_phi1 = df_loaded["phi_1merged"].iloc[min_F_index]
        min_phi2 = df_loaded["phi_2merged"].iloc[min_F_index]
        min_F = df_loaded["F"].iloc[min_F_index]
    
        # Compute the Euclidean distance between these points
        c1 = (initial_merged_concentration_guess[0] - min_phi1)
        c2 = (initial_merged_concentration_guess[1] - min_phi2)
        c3 = ((1-initial_merged_concentration_guess[0]-initial_merged_concentration_guess[1]) - (1-min_phi1-min_phi2))
        d = np.sqrt(c1**2 + c2**2 + c3**2)
        ds.append(d)

    # print(ds)
    fig, ax = plt.subplots(figsize=(4, 3))
    ax.plot(Xs, ds, linewidth = 2, color = "k")
    ax.set_xlabel("X")
    ylabel = r"$\sqrt{\Phi^2_{\text{guess}}-\Phi^2_{\text{minimum}}}$"
    ax.set_ylabel(ylabel)

    fig.tight_layout()

    output_filepath = f"data/noVolFluctuations/phi_g{phi_global}/analysis/"
    output_filename = f"XvsD.png"
    if not os.path.exists(output_filepath):
        os.makedirs(output_filepath)
    output_file = os.path.join(output_filepath, output_filename)

    plt.savefig(output_file, dpi=400)
    plt.close()

In [20]:
# saving dataframes for the 3 phase system

# A new system for each X
for X in tqdm(Xs):
    # Interaction matrix
    chis = np.array([[0, 3.0, 3+X], [3.0, 0.0, 3.0], [3+X, 3.0, 0.0]], dtype = DTYPE)
    # print(chis)
    
    # Equilibrium 3 phase solution
    phases = flory.find_coexisting_phases(3, chis, phi_global, progress=False)
    
    # print("\n" + f"Equlibrium phase volumes: {phases.volumes}")
    # print("Equlibrium phase concentrations:\n" + f"{phases.fractions}")
    
    # Take the transpose as the notation is different:
    # For me: phi_ij = component i in compartment j
    # For flory: phi_ji = component i in compartment j (OR I THINK SO!)
    
    vols = phases.volumes
    
    phases.fractions = np.transpose(phases.fractions)
    concs = phases.fractions

    # Compute the system's phase separated Free Energy'
    # Free energy of the 3 phase system
    
    F_flory = 0
    for i in range(len(phases.volumes)):
        F_flory += phases.volumes[i]*floryHuggins(phases.fractions[:, i], chis)
    
    # print(F_flory)
    output_filepath = f"data/withVolFluctuations/phi_g{phi_global}/raw/X{X:.3f}/"
    output_filename = f"FLORY_3phase_solution.pkl"
    if not os.path.exists(output_filepath):
        os.makedirs(output_filepath)
    output_file = os.path.join(output_filepath, output_filename)
    
    with gzip.open(output_file, "wb") as f:
        pickle.dump({
            "chis": chis,
            "phi_global": phi_global,
            "phase_fractions": concs,
            "phase_volumes": vols,
            "flory_free_energy": F_flory,
            "_metadata": {"chis-> the interaction matrix",
                          "phi_global-> global concentration of components",
                          "phase_fractions-> concentration of components in each compartment after running FLory",
                          "phase_volumes-> volumes of each compartment after running Flory",
                          "flory_free_energy-> free energy of the 3 componenet, 3 compartment solution (outputted from Flory)"
                         }
        }, f, protocol=pickle.HIGHEST_PROTOCOL)

100%|██████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.66it/s]


In [34]:
# Find the Best Merger in 2 steps:
# 1. Find the best volume slice for a specific merger from [12] [23] [13]
# 2. Then Find the best merger from these best volume slices

for X in (Xs):
    merged_compartments_list = [np.array([1, 2], dtype=np.int64), np.array([2, 3], dtype=np.int64), np.array([1, 3], dtype=np.int64)]
    # merged_compartments_list = [np.array([1, 2], dtype=np.int64), np.array([2, 3], dtype=np.int64)]

    for idx, merged_compartments in enumerate(merged_compartments_list):
        eta_merged_perturbed = np.linspace(max(tol, eta_merged_initial_guesses[idx] - n_points*step_size), min(1-tol, eta_merged_initial_guesses[idx] + n_points*step_size), 2*n_points+1)

        # Find the best volume slice
        min_F_best = DTYPE(np.inf)
        best_slice = None
        
        for eta_m in eta_merged_perturbed:
            input_filepath = f"data/withVolFluctuations/phi_g{phi_global}/raw/X{X:.3f}/mergers/{merged_compartments}/volume_slices/"
            input_filename = f"eta_m{eta_m:.3f}.pkl"
            input_file = os.path.join(input_filepath, input_filename)
            
            with gzip.open(input_file, "rb") as f:
                loaded_data = pickle.load(f)
                
            print(np.min(loaded_data["F"]))
        print()
        print()

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
-0.04719494431218257
-0.04722299865285549
-0.047249984893638866
-0.047271097060302546
-0.047293776345928656
-0.04730952058852924
-0.04732634634322011
-0.04733768379769405
-0.04734716288147909
-0.047354251838782585
-0.04735589889316598
-0.0473587003333307
-0.04735389943895604
-0.04735066946866647
-0.04734136480388235
-0.047329773066906736
-0.0473158922838797
-0.04729652072371946
-0.047279123398832945
-0.04725367047792391
-0.04723008262485955
-0.04719994843687471
-0.04716723804646226
-0.04713230859578307
-0.0470925046922952
-0.04705475005244539
-0.0470083433265856
-0.

In [30]:
np.min(loaded_data["F"])

nan

In [21]:
# Saving dataframes for each compartment
for X in tqdm(Xs):
    # Get the 3 phase solution
    input_filepath = f"data/withVolFluctuations/phi_g{phi_global}/raw/X{X:.3f}/"
    input_filename = f"FLORY_3phase_solution.pkl"
    input_file = os.path.join(input_filepath, input_filename)
    
    with gzip.open(input_file, 'rb') as f:
        loaded_data = pickle.load(f)

    concs = loaded_data["phase_fractions"]
    vols = loaded_data["phase_volumes"]
    chis = loaded_data["chis"]
    
    merged_compartments_list = [np.array([1, 2], dtype=np.int64), np.array([2, 3], dtype=np.int64), np.array([1, 3], dtype=np.int64)]
    # merged_compartments_list = [np.array([1, 2], dtype=np.int64)]
    
    eta_merged_initial_guesses = [] # STore the initial eta_merged guesses for constructing the perturbed eta array while looking for the best merger
    
    for merged_compartments in merged_compartments_list:
        # print(merged_compartments)
        
        phi_in_kunmerged, phi_in_kmerged, eta_merged, unmerged_compartment = mergers(concs, vols, chis, merged_compartments)
        eta_merged_initial_guesses.append(eta_merged)
        # print(phi_in_kunmerged)
        # print(phi_in_kmerged)
        # print(merged_compartments, eta_merged)
        # print(unmerged_compartment)
        
        # Save the volume slices
        perturb_mergers(concs, vols, chis, phi_in_kunmerged, phi_in_kmerged, eta_merged, merged_compartments, unmerged_compartment, step_size, n_points)
        
# Accessing data
# with gzip.open(output_file, 'rb') as f:
#     loaded_data = pickle.load(f)

# df_loaded = loaded_data['dataframe']
# eta_merged_loaded = loaded_data['metadata']['eta_merged']

/tmp/ipykernel_148903/888854170.py:3: RuntimeWarning: invalid value encountered in log0<?, ?it/s]
  part_1 = np.sum(phi*np.log(phi))
100%|█████████████████████████████████████████████████████████████| 1/1 [03:15<00:00, 195.18s/it]


In [ ]:
# Finding the best merger
for X in tqdm(Xs):
    merged_compartments_list = [np.array([1, 2], dtype=np.int64), np.array([2, 3], dtype=np.int64), np.array([1, 3], dtype=np.int64)]

    min_F_best = DTYPE(np.inf)
    best_merger = None
    
    for merged_compartments in merged_compartments_list:
        input_filepath = f"data/noVolFluctuations/phi_g{phi_global}/raw/X{X:.3f}/mergers/"
        input_filename = f"{merged_compartments}.pkl"
        input_file = os.path.join(input_filepath, input_filename)
    
        with gzip.open(input_file, "rb") as f:
            loaded_data = pickle.load(f)
        
        df_loaded = loaded_data["dataframe"]
        # print(df_loaded)
        min_F_merger_index = np.argmin(df_loaded["F"])
        min_F_merger = df_loaded["F"].iloc[min_F_merger_index]

        if min_F_merger <= min_F_best:
            min_F_best = min_F_merger
            best_merger = merged_compartments
        # print(best_merger)

        output_filepath = f"data/noVolFluctuations/phi_g{phi_global}/analysis/X{X:.3f}"
        output_filename = f"best_merger.pkl"
        if not os.path.exists(output_filepath):
            os.makedirs(output_filepath)
        output_file = os.path.join(output_filepath, output_filename)
        
        data_to_save = {
            "best_merger": best_merger,
            "metadata": "stores the compartments which on getting merged return locally the lowest free energy"
            }
        
        with gzip.open(output_file, "wb") as f:
            pickle.dump(data_to_save, f, protocol=pickle.HIGHEST_PROTOCOL)
    # print(best_merger)

In [ ]:
# Xs = np.arange(0, 10.1, 0.2)
plot_XvsD(Xs)
# plot_Contours(Xs)
plot_FvsX(Xs)
